# Endevina l'animal

En aquesta pràctica implementarem l'exemple de sistema expert vist a teoria, on segons unes característiques donades, el sistema expert ha d'endevinar quin animal és. En aquest cas, el sistema expert només tindrà en compte els animals que es mostren a l'esquema de teoria, és a dir, els animals que es mostren a la següent imatge:

<br />
<div>
<img src="https://lawer.github.io/mia/apunts/images%2FAND-OR-Tree.png" width="900"/>
</div>

Utilitzarem el mòdul de python `experta` per implementar un sistema expert **forward chaining**. Aquest mòdul ens permetrà crear un sistema expert de manera molt senzilla, ja que només haurem de definir les regles i els fets que utilitzarà el nostre sistema expert. 

Si implimentarem nosaltres un motor d'inferència tindriem problemes de rendiment quan el sistema expert tingués moltes regles, ja que hauríem de comprovar totes les regles per a cada fet. En canvi, `experta` té una implementació molt optimitzada (utilitzant l'algorisme RETE) que ens permetrà crear sistemes experts amb moltes regles i fets.

A continuació, instal·larem i importarem les llibreries necessàries per a la pràctica.

In [19]:
!pip install git+https://github.com/openmotics/om-experta.git

  Cloning https://github.com/openmotics/om-experta.git to /private/var/folders/zm/06zm__c5637bs4z8mhp4pfph0000gn/T/pip-req-build-b_agizuu
  Running command git clone --filter=blob:none --quiet https://github.com/openmotics/om-experta.git /private/var/folders/zm/06zm__c5637bs4z8mhp4pfph0000gn/T/pip-req-build-b_agizuu
  Resolved https://github.com/openmotics/om-experta.git to commit d35d53708a46482e1ee4e3a4bc1a36bc03492913
  Preparing metadata (setup.py) ... done

[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [20]:
from experta import *

## Definició del sistema expert

Definirem el nostre sistema expert com una classe que hereta de `KnowledgeEngine`. Cada regla es defineix mitjançant una funció amb l'anotació `@Rule`, que especifica quan s'ha d'executar la regla. A dins de la regla, podem afegir nous fets mitjançant la funció `declare`, i afegir aquests fets provocarà que s'executin més regles mitjançant el motor d'inferència.

El contingut de `@Rule` és una expressió lògica que s'avalua a `True` o `False`. Aquesta expressió lògica pot contenir **fets**, operadors lògics (`AND`, `OR`, `NOT`) i operadors de comparació (`==`, `!=`, `<`, `>`, `<=`, `>=`). Els fets són objectes de la classe `Fact`, que poden tenir atributs que s'especifiquen com a paràmetres de la classe `Fact`. Per exemple, el fet `Fact('mamífer', pel=True)` té l'atribut `pel` amb valor `True`.

Adicionalment, podem utilitzar la funció `MATCH` per a especificar que un atribut pot tenir qualsevol valor. Per exemple, `Fact('mamífer', pel=MATCH.pel)` és un fet que té l'atribut `pel` amb qualsevol valor.

In [21]:
class Animals(KnowledgeEngine):
    # Regles    
    @Rule(OR(
        AND(Fact('dents afilades'), Fact('ungles'), Fact('ulls mirant endavant')),
        Fact('carnivor')))
    # Si l'animal té dents afilades, ungles i ulls mirant endavant, o menja carn, llavors és un carnivor
    def carnivor(self):
        # Afegim el fet 'carnivor' al sistema expert
        self.declare(Fact('carnivor'))

    @Rule(OR(Fact('pel'), Fact('dona llet')))
    # Si l'animal té pèl o dóna llet, llavors és un mamífer
    def mamifer(self):
        self.declare(Fact('mamifer'))

    @Rule(Fact('mamifer'),
          OR(Fact('te peulles'), Fact('mastega el bolus')))
    # Si l'animal és un mamifer i té peülles o mastega el _bolus_, llavors es un ungulat
    def peulles(self):
        self.declare('ungulat')

    @Rule(OR(Fact('plomes'), AND(Fact('vola'), Fact('pon ous'))))
    # Si l'animal té plomes o vola i pon ous, llavors es un ocell
    def ocell(self):
        self.declare('ocell')

    @Rule(Fact('mamifer'), Fact('carnivor'),
          Fact(color='marro-vermellos'),
          Fact(pattern='taques fosques'))
    # Si l'animal és un mamifer carnivor, de color marró vermellós i amb taques fosques, llavors és un mico
    def monkey(self):
        self.declare(Fact(animal='mico'))

    @Rule(Fact('mamifer'), Fact('carnivor'),
          Fact(color='marro-vermellos'),
          Fact(pattern='ratlles fosques'))
    # Si l'animal es un mamifer carnivor, de color marró vermellós i amb ratlles fosques, llavors és un tigre
    def tigre(self):
        self.declare(Fact(animal='tigre'))

    @Rule(Fact('ungulat'),
          Fact('coll llarg'),
          Fact('long legs'),
          Fact(pattern='taques fosques'))
    # Si l'animal és un ungulat, de coll llarg, potes llargues i amb taques fosques, llavors és una girafa
    def girafa(self):
        self.declare(Fact(animal='girafa'))

    @Rule(Fact('ungulat'),
          Fact(pattern='ratlles fosques'))
    # Si l'animal es un ungulat amb ratlles fosques, llavors es una zebra
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('ocell'),
          Fact('coll llarg'),
          Fact('no vola'),
          Fact(color='blanc i negre'))
    # Si l'animal es un ocell, de coll llarg, no vola i de color blanc i negre, llavors és un estruç
    def estruc(self):
        self.declare(Fact(animal='estruc'))

    @Rule(Fact('ocell'),
          Fact('nada'),
          Fact('no vola'),
          Fact(color='blanc i negre'))
    # Si l'animal es un ocell, nada, no vola i de color blanc i negre, llavors es un pingüí
    def pingui(self):
        self.declare(Fact(animal='pingui'))

    @Rule(Fact('ocell'),
          Fact('vola be'))
    # Si l'animal es un ocell i vola bé, llavors es un albatros
    def albatros(self):
        self.declare(Fact(animal='albatros'))

    @Rule(Fact(animal=MATCH.a))
    # Si està definit l'animal, llavors el mostrem
    def print_result(self, a):
        print(f"L'animal és un {a}")

    def agregar_fets(self, fets):
        # Afegim els fets passats com a paràmetre (fets) al sistema expert
        # Utilitzarem aquesta funció per a inicialitzar el sistema expert
        for f in fets:
            self.declare(f)

# Testeig del sistema expert

Una vegada hem definit la base de coneixement, inicialitzem la memòria de treball amb uns fets inicials, i després cridem el mètode `run()` per a realitzar la inferència. Podem veure com a resultat que s'afegeixen nous fets inferits a la memòria de treball, incloent-hi el fet final sobre l'animal (si hem configurat tots els fets inicials correctament).

Anem a veure un exemple on inicialitzem el sistema expert amb els fets `pel`, `dents afilades`, `ungles` i `ulls mirant endavant`. Aquests fets són suficients per a inferir que l'animal és un carnivor.


In [22]:
# Creem una instància del nostre sistema expert
animals = Animals()

# Reset del sistema expert. Necessari per a tornar a l'estat inicial.
animals.reset()

# Afegim els fets inicials.
animals.agregar_fets(
    [Fact('pel'), Fact('dents afilades'), Fact('ungles'), Fact('ulls mirant endavant')]
)

# Executem el sistema expert
animals.run()

animals.facts

FactList([(0, InitialFact()),
          (1, Fact('pel')),
          (2, Fact('dents afilades')),
          (3, Fact('ungles')),
          (4, Fact('ulls mirant endavant')),
          (5, Fact('carnivor')),
          (6, Fact('mamifer'))])

Podem veure com s'ha inferit el fet `carnivor` a partir dels fets inicials, i com s'ha inferit el fet `mamifer` a partir del fet `pel`. 

Anem a agregar els fets `marro-vermellos` primer i `taques fosques` després. Veurem com primer no s'infereix cap animal, i després s'infereix el fet `mico` a partir dels fets `mamifer`, `carnivor`, `marro-vermellos` i `taques fosques`.

In [23]:
animals.agregar_fets(
    [Fact(color='marro-vermellos')]
)
animals.run()
animals.facts

FactList([(0, InitialFact()),
          (1, Fact('pel')),
          (2, Fact('dents afilades')),
          (3, Fact('ungles')),
          (4, Fact('ulls mirant endavant')),
          (5, Fact('carnivor')),
          (6, Fact('mamifer')),
          (7, Fact(color='marro-vermellos'))])

In [24]:
animals.agregar_fets(
    [Fact(pattern='taques fosques')]
)
animals.run()
animals.facts

L'animal és un mico


FactList([(0, InitialFact()),
          (1, Fact('pel')),
          (2, Fact('dents afilades')),
          (3, Fact('ungles')),
          (4, Fact('ulls mirant endavant')),
          (5, Fact('carnivor')),
          (6, Fact('mamifer')),
          (7, Fact(color='marro-vermellos')),
          (8, Fact(pattern='taques fosques')),
          (9, Fact(animal='mico'))])